In [1]:
sparkSql = (SparkSession.builder
         .master("local")
         .appName("Kiva Exploration")
         .getOrCreate())

loans = sparkSql.read.format('json').load('kiva-data/loans.json')
loans.registerTempTable('loans')
lenders = sparkSql.read.format('json').load('kiva-data/lenders.json')
lenders.registerTempTable('lenders')
loans_lenders = sparkSql.read.format('json').load('kiva-data/loans_lenders.json')
loans_lenders.registerTempTable('loans_lenders')

# Custom Functions

## Gender Ratio

0 = All female

1 = All male

In [2]:
import pyspark

def gender_ratio(array):
    num_males = 0
    for item in array:
        if item.gender == 'M':
            num_males += 1
            
    return float(num_males) / len(array)

sparkSql.udf.register('gender_ratio',
                      gender_ratio,
                      pyspark.sql.types.FloatType())

## Fetch GDP

In [3]:
import pandas as pd
from datetime import datetime
import numpy as np


# Load country info data
country_codes_raw = pd.read_csv('economic-data/country-codes.csv')
country_gdp_raw = pd.read_csv('economic-data/country-gdp.csv')

# Clean country codes data
country_codes = country_codes_raw[['official_name_en', 'ISO3166-1-Alpha-2', 
                                   'ISO3166-1-Alpha-3', 'ISO4217-currency_alphabetic_code']]

# Clean gdp data
country_gdp = country_gdp_raw.drop(country_gdp_raw.columns[[0, 1]], axis=1)
country_gdp.columns = ['name', 'country_code_3', '2002', '2003', '2004', '2005', '2006',
                       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

# Merge gdp and code
country_gdp = pd.merge(country_gdp, country_codes, left_on='country_code_3', right_on='ISO3166-1-Alpha-3', how='left')
country_gdp.drop(['official_name_en', 'ISO3166-1-Alpha-3', 'country_code_3'], axis=1, inplace=True)
country_gdp = country_gdp.rename(columns = {'ISO3166-1-Alpha-2':'country_code',
                                            'ISO4217-currency_alphabetic_code':'currency_code'})
country_gdp.replace('..', np.nan, inplace=True)

# Reorder columns
cols = list(country_gdp.columns)
cols.insert(1, cols.pop(cols.index('country_code')))
cols.insert(2, cols.pop(cols.index('currency_code')))
country_gdp = country_gdp.reindex(columns= cols)

def gdp(country_code, disbursal_date):
    def historical_gdp(array):
        array = np.array(map(float, array))
        array = array[~np.isnan(array)] # Remove NaN
        if len(array) == 0: # No GDP values
            return 0
        return float(np.mean(array, dtype=np.float64))
        
    # TODO: Unable to resolve country code WorldBank dataset has wrong alpha 3 codes e.g. Andorra causing issues
    try:
        float(country_code)
        return 0
    except:
        if country_code not in list(country_gdp['country_code']):
            return 0 # TODO: Bad solution ?   
    
    # Get the historical average GDP if no disbursal date
    all_gdp = country_gdp[country_gdp.country_code == country_code].values[0][3:]
    if (disbursal_date is None): # or (country_gdp[date][country_gdp.country_code == country_code] == float('Nan')):
        return historical_gdp(all_gdp)
    
    date = str(datetime.strptime(disbursal_date, '%Y-%m-%dT%H:%M:%SZ').year)
    # Get the historical average GDP if no GDP for that year
    if pd.isnull(country_gdp[date][country_gdp.country_code == country_code].values[0]):
        return historical_gdp(all_gdp)
    
    return float(country_gdp[date][country_gdp.country_code == country_code].values[0])

sparkSql.udf.register('gdp', gdp, pyspark.sql.types.FloatType())

## Fetch Exchange Rates

In [4]:
currencies_raw = pd.read_csv('economic-data/currencies.csv')
# Cleanup
currencies = currencies_raw.drop(country_gdp_raw.columns[[0, 1]], axis=1)
currencies.columns = ['country_name', 'country_code_3', '2002', '2003', '2004', '2005', '2006',
                       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

# Get ISO 2 code
currencies = pd.merge(currencies, country_codes, left_on='country_code_3', right_on='ISO3166-1-Alpha-3', how='left')
currencies.drop(['official_name_en', 'ISO3166-1-Alpha-3', 'country_code_3'], axis=1, inplace=True)
currencies = currencies.rename(columns = {'ISO3166-1-Alpha-2':'country_code',
                                            'ISO4217-currency_alphabetic_code':'currency_code'})
currencies.replace('..', np.nan, inplace=True)

# Add code for European Union
currencies.set_value(217, 'country_code', 'EU')
currencies.set_value(217, 'currency_code', 'EMU')

# Reorder columns
cols = list(currencies.columns)
cols.insert(1, cols.pop(cols.index('country_code')))
cols.insert(2, cols.pop(cols.index('currency_code')))
currencies = currencies.reindex(columns=cols)

def xchange_rate(country_code, disbursal_date):
    def historical_rates(array):
        array = np.array(map(float, array))
        array = array[~np.isnan(array)] # Remove NaN
        if len(array) == 0: # No rate values
            return 1
        return float(np.mean(array, dtype=np.float64))
    
    eu = ['AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','GR','HU','IE',
          'IT','LV','LT','LU','MT','NL','PL','PT','RO','SK','SI','ES','SE','GB']
    us = ['AS','GU','MP','PR','UM','VI']
    try:
        float(country_code) # Country code unknown?
        if pd.isnull(country_code):
            return 1 # TODO: Bad solution ??
    except:
        if country_code in eu:
            country_code = 'EU'
        elif country_code in us:
            country_code = 'US'
        if country_code not in list(currencies['country_code']):
            return 1
        
        
    # TODO: Unable to resolve country code WorldBank dataset has wrong alpha 3 codes e.g. Andorra causing
    try:
        float(country_code)
        return 0
    except:
        if country_code not in list(currencies['country_code']):
            return 0 # TODO: Bad solution    
    
    # Get the historical average exchange rate if no disbursal date
    all_rates = currencies[currencies.country_code == country_code].values[0][3:]
    if (disbursal_date is None): # or (country_gdp[date][country_gdp.country_code == country_code] == float('Nan')):
        return historical_rates(all_rates)
    
    date = str(datetime.strptime(disbursal_date, '%Y-%m-%dT%H:%M:%SZ').year)
    # Get the historical average exchange rate if no GDP for that year
    if pd.isnull(currencies[date][currencies.country_code == country_code].values[0]):
        return historical_rates(all_rates)
    
    return float(currencies[date][currencies.country_code == country_code].values[0])

sparkSql.udf.register('xchange_rate', xchange_rate, pyspark.sql.types.FloatType())

# Fetch actual data

Get all data that we are going to use, get dummies, then split into train/validation/test.

Query our datasets to train on.

In [6]:
query = '''
SELECT
    id,
    activity,
    size(borrowers) as num_borrowers,
    gender_ratio(borrowers) as gender_ratio,
    lender_count,
    location.country,
    location.country_code,
    partner_id,
    sector,
    tags,
    DATEDIFF(terms.disbursal_date, planned_expiration_date) as loan_length,
    terms.disbursal_amount,
    terms.disbursal_currency,
    terms.disbursal_date,
    size(terms.scheduled_payments) as num_repayments,
    terms.repayment_interval,
    CASE WHEN
        (status = 'refunded') OR
        (status = 'defaulted') OR
        (status = 'deleted') OR
        (status = 'issue') OR
        (status = 'inactive_expired') OR
        (status = 'expired') OR
        (status = 'inactive') OR
        (delinquent = True) THEN 1 ELSE 0 END AS bad_loan,
    gdp(location.country_code, terms.disbursal_date) as gdp,
    xchange_rate(location.country_code, terms.disbursal_date) as xchange_rate,
    status,
    delinquent
    
FROM loans
WHERE
    status != 'fundraising' AND
    status != 'funded'
'''

dataset = sparkSql.sql(query).toPandas()

# Data Splits

In [8]:
X_columns = [
    'activity', 'num_borrowers', 'gender_ratio',
    'lender_count', 'country', 'partner_id', 'sector',
    'loan_length', 'disbursal_amount', 'disbursal_currency',
    'num_repayments', 'repayment_interval', 'gdp', 'xchange_rate'
]

y_column = ['bad_loan']

dummy_set = pd.get_dummies(dataset[X_columns + y_column])
dummy_set.to_csv('processed_dummy.csv')

Now we can restart the kernel to clear memory, and start processing.

In [1]:
import pandas as pd

processed_dummy = pd.read_csv('processed_dummy.csv', index_col=0)

In [3]:
import numpy as np

train, validate, test = np.split(processed_dummy.sample(frac=1, random_state=0),
                                 [int(.6*len(processed_dummy)),
                                  int(.8*len(processed_dummy))])

train.to_csv('processed_train.csv')
validate.to_csv('processed_validate.csv')
test.to_csv('processed_test.csv')

# Testing all the models

In [1]:
import pandas as pd
train = pd.read_csv('processed_train.csv', index_col=0).dropna(axis=1)
valid = pd.read_csv('processed_validate.csv', index_col=0).dropna(axis=1)

In [2]:
len(train.columns)

342

Naive guess:

In [2]:
train_x = train.drop('bad_loan', axis=1)
train_y = train['bad_loan']
valid_x = valid.drop('bad_loan', axis=1)
valid_y = valid['bad_loan']

1 - train_y.mean()

0.89836166750827584

SVM

In [ ]:
from itertools import product
import pickle
from sklearn.svm import SVC

svc_params = product([1, .5, 1.5], [.001, .01, .1])

for C, gamma in svc_params:
    svc = SVC(C=C, gamma=gamma)

    svc.fit(train_x, train_y)
    with open('svc_{}_{}.pickle'.format(C, gamma), 'w') as handle:
        pickle.dump(svc, handle)
        
    print("C: {}; gamma: {}; score: {}".format(
            C, gamma, svc.score(train_x, train_y)))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Number of columns is 342
for n_components in [342, 250, 150, 75]
    lda = LinearDiscriminantAnalysis(n_components=n_components)
    lda.fit(train_x, train_y)
    with open('lda_{}.pickle'.format(n_components), 'w') as handle:
        pickle.dump(lda, handle)
    
    print("N_components: {}; score: {}".format(
            n_components, lda.score(valid_x, valid_y)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

for n_estimators in [10, 50, 75, 100]:
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(train_x, train_y)
    with open('rf_{}.pickle'.format(n_estimators), 'w') as handle:
        pickle.dump(rf, handle)
    
    print("N_estimators: {}; score: {}".format(
            n_estimators, score(valid_x, valid_y)))